In [ ]:
# Magic functions -- Run Once
%load_ext autoreload
%autoreload 2
%matplotlib notebook

# Move up one folder to reach the repo root
%cd ..

from utils.notebook.generic import full_width_notebook

full_width_notebook()

In [ ]:
# Paths, Imports & Configs
import matplotlib.pyplot as plt
import pandas as pd
from copy import deepcopy
from utils.notebook.experiment_explorer import get_experiments, get_format_dicts

from utils.notebook.pandas import sub_cols_with_cond_and_create_new_col, grouped_scatter, groupby_mean, convert_cols_to_int

root_data_path = "data"
#root_output_path = "output/training"
root_output_path = "output_synced/training"
#root_output_path = "output/synced_training"

# Filtering option
latest_experiment_notes = ['extractor_original', 'extractor_original_512_win']

# Helpers column selectors
run_info_columns = ['nb_sample', 'config', 'nb_epoch_trained', 'stopped_early']
more_accuracy_info_columns = ['0.6_at_epoch', '0.7_at_epoch', '0.8_at_epoch', '0.9_at_epoch']
accuracy_columns = ['best_val_acc', 'train_acc', 'test_acc']
loss_columns = ['best_val_loss', 'train_loss', 'test_loss']
all_training_stats = ['all_train_acc', 'all_train_loss', 'all_val_acc', 'all_val_loss']
model_config_columns = ['nb_trainable_param', 'word_embedding_dim', 'rnn_state_size', 'extractor_type', 
                        'stem_out_chan', 'nb_resblock', 'resblocks_out_chan', 'classifier_conv_out_chan', 
                        'classifier_type', 'classifier_global_pool', 'nb_answer']
optimizer_columns = ['optimizer_type', 'optimizer_lr', 'optimizer_weight_decay', 'dropout_drop_prob']
preprocessing_columns = ['pad_to_largest', 'resized_height', 'resized_width']
spectrogram_columns = ['n_fft', 'hop_length', 'keep_freq_point', 'n_mels', 'resample_audio']
normalization_columns = ['norm_zero_one', 'norm_clear_stats']
timing_columns = ['train_time', 'mean_epoch_time']
other_columns = ['prefix', 'nb_scene', 'nb_q_per_scene', 'date', 'batch_size', 'resnet_features', 'test_version', 
                 'random_seed', 'nb_non_trainable_param', 'total_nb_param', 'nb_epoch', 'stop_accuracy', 'folder', 'gpu_name']

audio_cols = run_info_columns + accuracy_columns + spectrogram_columns + normalization_columns + ['random_seed', 'nb_trainable_param_round', 'note']
exp_results_cols = run_info_columns + ['nb_scene', 'nb_q_per_scene'] + accuracy_columns + more_accuracy_info_columns + ['nb_trainable_param_round', 'note']

# Retrieve all experimentsi infos
experiments = get_experiments(root_output_path, question_type_analysis=False)

# Pretty printing
format_dict, latex_format_dict = get_format_dicts()

pd.set_option('display.max_colwidth', -1)

exp_results = experiments[exp_results_cols].sort_values('test_acc', ascending=False)
audio_results = experiments[audio_cols].sort_values('test_acc', ascending=False)

latest_experiments = audio_results[audio_results['note'].isin(latest_experiment_notes)]

# Print available columns
experiments.columns.values


In [ ]:
latest = experiments[experiments['date'] >= '2020-09-12'].sort_values('date', ascending=False)
latest[audio_cols + ['resized_width', 'resized_height', 'extractor_type', 'date', 'train_time']].style.format(format_dict)

l = latest[latest['rnn_state_size'].isin([256,512,2048])]
l = l[['date', 'random_seed', 'config', 'note', 'rnn_state_size', 'extractor_type', 'extractor_filters', 'extractor_projection_size', 'nb_resblock', 'resblocks_out_chan', 'classifier_conv_out', 'classifier_projection_out', 'train_acc', 'best_val_acc', 'test_acc']]
display(l.sort_values('note', ascending=False).style.format(format_dict))
#display(l.style.format(format_dict))
l['config'].unique()

In [ ]:
latest = experiments[experiments['date'] >= '2020-09-12'].sort_values('date', ascending=False)
#latest[latest['nb_epoch_trained'] < 15][['config', 'test_acc','nb_epoch_trained']]
latest[latest['note'].str.contains('smaller_2d_extractor')][['config','extractor_type','note']]

In [ ]:
experiments[experiments['note'].str.contains('table_4_classifier_topologies_1_worker', na=False)]['train_time'].mean()

In [ ]:
from utils.notebook.experiment_explorer import get_full_sync_experiment_from_drive_script

columns = ['mean_epoch_time', 'config', 'random_seed', 'input_type', 'extractor_type', 'train_acc', 'best_val_acc', 'test_acc', 'all_train_acc', 'date']
reproduc = experiments[experiments['note'].str.contains("REPRODUC_TEST", na=False) & (~experiments['input_type'].str.contains('raw_h5'))]

reproduc[columns].style.format(latex_format_dict)

#print("\n".join(get_full_sync_experiment_from_drive_script(reproduc, "output_synced/training", dryrun=True)))

In [ ]:
from utils.notebook.pandas import color_by_multi_attribute
#filters = (experiments['resized_height'].notna())
#filters = (experiments['extractor_type'].str.contains('ConvLearned'))
#filters = (experiments['config'].str.contains('film_original'))
#filters = (experiments['nb_trainable_param'] < 800000)
#filters = ((experiments['nb_resblock'] == 4) & (experiments['resblocks_out_chan'] == 128))
#filters &= ((experiments['n_fft'] == 512))
#filters = (experiments['input_type'].str.contains('RGB'))
filters = (experiments['date'] >= '2020-09-12')

audio_exp = experiments[filters].sort_values('test_acc', ascending=False)
#audio_exp = experiments[filters].sort_values('date', ascending=False)

cols = ['config', 'n_fft', 'nb_resblock', 'resblocks_out_chan', 'resized_height', 'stem_spatial_location', 'resblock_spatial_location', 'classifier_spatial_location', 'train_acc', 'best_val_acc', 'test_acc', 'extractor_type', 'nb_trainable_param', 'random_seed', 'max_freq', 'spectrogram_rgb', 'batch_size', 'input_type', 'preprocessed_folder_name']
color_by_multi_attribute(audio_exp[cols], 'test_acc', ['stem_spatial_location', 'resblock_spatial_location', 'classifier_spatial_location', 'resized_height', 'extractor_type', 'nb_trainable_param'], cmaps=['Reds', 'ocean', 'ocean', 'ocean'], format_dict=latex_format_dict)

In [ ]:
len(experiments.index)

In [ ]:
format_dict, latex_format_dict = get_format_dicts()

In [ ]:
from utils.notebook.pandas import color_by_multi_attribute
#audio_exp = experiments[experiments['input_type'] == 'audio'].sort_values('date', ascending=False)
filters = (~experiments['stem_spatial_location'].str.contains('Both'))
filters &= (~experiments['input_type'].str.contains("RGB"))
#filters = (experiments['stem_spatial_location'].apply(lambda x : len(list(x)) <= 1))
#filters &= (experiments['resblock_spatial_location'].apply(lambda x : len(list(x)) <= 1))
#filters &= (experiments['classifier_spatial_location'].apply(lambda x : len(list(x)) <= 1))
#filters |= ((experiments['config'].isin(['extractor_slim_parallel_3_block_64_proj', 'extractor_slim_interleaved_3_block_64_proj'])) & (experiments['n_fft'] == 512) & (experiments['keep_freq_point'] == 256) & (experiments['resample_audio'].isnull()))
audio_exp = experiments[filters].sort_values('test_acc', ascending=False)
#audio_exp = experiments.sort_values('date', ascending=False)
cols = ['config', 'stem_spatial_location', 'resblock_spatial_location', 'classifier_spatial_location', 'train_acc', 'best_val_acc', 'test_acc', 'extractor_type', 'nb_trainable_param', 'random_seed', 'max_freq', 'spectrogram_rgb', 'batch_size', 'input_type', 'preprocessed_folder_name']
#audio_exp[cols].style.format(latex_format_dict)
color_by_multi_attribute(audio_exp[cols], 'test_acc', ['stem_spatial_location', 'resblock_spatial_location', 'classifier_spatial_location', 'extractor_type'], cmaps=['Reds', 'ocean', 'ocean', 'ocean'], format_dict=latex_format_dict)

In [ ]:
sorted(audio_exp['stem_spatial_location'].unique())

In [ ]:
latest = experiments[experiments['date'] >= '2020-09-12'].sort_values('date', ascending=False)
latest[audio_cols + ['extractor_type', 'date', 'train_time']].style.format(format_dict)
#latest.sort_values('test_acc', ascending=False)


In [ ]:
latest[latest['note'] == 'extractor_slim'].sort_values('test_acc', ascending=False)

In [ ]:
nfft_512_exps = latest[latest['n_fft'] == 512].sort_values('test_acc', ascending=False)
nfft_512_exps.style.format(format_dict)

In [ ]:
#from utils.notebook.pandas import color_row_by_attribute

import itertools
import random
from IPython.core.display import display, Markdown
from pandas.api.types import is_numeric_dtype

def color_row_by_attribute(df, attribute, cmap=None, format_dict=None):
    """
    Will color the rows based on the attribute values.
    Data should be sorted in the desired order before passing to this function
    This will return a Styler object.
    """

    def styler_fct(sample, attribute, min_max, categorical_values=None, cmap=None):

        if cmap is None:
            cmap = plt.get_cmap('Blues')

        val = sample[attribute]
        
        if pd.isnull(val):
            val = 0
        
        if categorical_values:
            val = categorical_values[val]
            
        # Will generate a value in the range of 20,220
        val = int(((val - min_max[0]) / (min_max[1] - min_max[0])) * 200 + 20)

        color = tuple([int(c*255) for i, c in enumerate(cmap(val)) if i < 3])
        css = f"background-color: rgb{color}"
        return [css] * len(sample.index)
    
    # Should we fill with None ?
    df_copy = df.fillna(value=0)
    
    # What happend when there is a none value in the column ? What about Nn ?
    if not is_numeric_dtype(df_copy[attribute]):
        # Create range from 0 to 1 for categorical data
        unique_values = df_copy[attribute].unique()
        nb_values = len(unique_values)
        
        categorical_values = {value: i/nb_values for i, value in enumerate(unique_values)}
        min_max = (0, 1)
    else:
        min_max = df_copy[attribute].min(), df_copy[attribute].max()
        categorical_values = None

    styler = df_copy.style.apply(styler_fct, 
                           attribute=attribute, 
                           min_max=min_max,
                           categorical_values=categorical_values,
                           cmap=cmap, 
                           axis=1)
    
    if format_dict:
        styler = styler.format(format_dict)
    
    return styler


def color_by_multi_attribute(df, main_attribute, attributes=None, cmaps=None, format_dict=None, print_infos=True):
    """
    Will color the whole rows based on the main_attribute values.
    Will color specific columns based on attributes values (Over the main_attribute color)
    Data should be sorted in the desired order before passing to this function
    This will return a Styler object.
    """
    
    # Color map handling
    if cmaps is None:
        cmaps = ['Blues']
    elif type(cmaps) != list:
        cmaps = [cmaps]
        
    for i, cmap in enumerate(cmaps):
        if type(cmap) == str:
            cmaps[i] = plt.get_cmap(cmap)
            
    main_attribute_cmap = cmaps[0]
    other_attributes_cmaps = cmaps[1:]

    # Styler fct
    def main_attribute_styler_fct(sample, attribute, min_max, categorical_values=None, cmap=None):
        val = sample[attribute]
        
        if pd.isnull(val):
            val = 0
        
        if categorical_values:
            val = categorical_values[val]
            
        # Will generate a value in the range of 20,220
        val = int(((val - min_max[0]) / (min_max[1] - min_max[0])) * 200 + 20)

        color = tuple([int(c*255) for i, c in enumerate(cmap(val)) if i < 3])
        css = f"background-color: rgb{color}"
        return [css] * len(sample.index)
    
    # Filling NaN values so their value can be expressed as color with cmap
    df_copy = df.fillna(value=0)
    
    # Prepare data normalization
    if not is_numeric_dtype(df_copy[main_attribute]):                                   # FIXME : Or nb unique value < X ?
        # Create range from 0 to 1 for categorical data
        unique_values = df_copy[main_attribute].unique()
        nb_values = len(unique_values)
        
        categorical_values = {value: i/nb_values for i, value in enumerate(unique_values)}
        min_max = (0, 1)
    else:
        min_max = df_copy[main_attribute].min(), df_copy[main_attribute].max()
        categorical_values = None

    if print_infos:
        print(f"[MAIN] Highlighting '{main_attribute}' with cmap '{main_attribute_cmap.name}'")
        
    styler = df_copy.style.apply(main_attribute_styler_fct, 
                           attribute=main_attribute, 
                           min_max=min_max,
                           categorical_values=categorical_values,
                           cmap=main_attribute_cmap, 
                           axis=1)
    
    # Highlight specific columns over the main_attributes highlighting
    if attributes:
        if type(attributes) != list:
            attributes = [attributes]
        
        # We won't use reversed cmaps, if the user want it, he can manually specify it
        all_cmaps = [m for m in plt.colormaps() if '_r' not in m]
        
        # FIXME : Need something like if not is_numeric_dtype(df_copy[main_attribute])
        
        while len(other_attributes_cmaps) < len(attributes):
            new_cmap = plt.get_cmap(random.sample(all_cmaps, 1)[0])
            
            if new_cmap not in other_attributes_cmaps:
                other_attributes_cmaps.append(new_cmap)
                    
        for attribute, cmap in zip(attributes, other_attributes_cmaps):
            styler = styler.background_gradient(cmap=cmap, subset=[attribute])
            
            if print_infos:
                print(f"[SUB]  Highlighting colum '{attribute}' with cmap '{cmap.name}'")
    
    if format_dict:
        styler = styler.format(format_dict)
    
    return styler

In [ ]:
#exps = audio_exp.sort_values(['n_fft', 'test_acc'], ascending=[False, False])
exps = audio_exp.sort_values('test_acc', ascending=False)
columns = ['config', 'max_freq', 'test_acc', 'n_fft', 'keep_freq_point', 'hop_length','n_mels', 'resample_audio', 'extractor_type', 'nb_trainable_param_round', 'note']
exps = exps[columns]

grouped = exps.fillna(0).groupby(['config','max_freq','extractor_type', 'n_mels'], as_index=False).mean().sort_values('test_acc', ascending=False)
styler = color_by_multi_attribute(grouped, main_attribute='extractor_type', attributes=['n_mels', 'resample_audio', 'nb_trainable_param_round'], cmaps=['YlOrRd', 'RdBu'], format_dict=format_dict)

color_by = 'max_freq'
#color_by = 'resample_audio'
#color_by = 'n_fft'

display(Markdown(f"# Color coded by {color_by.capitalize()}"))
styler

In [ ]:
#exps = audio_exp.sort_values(['n_fft', 'test_acc'], ascending=[False, False])
exps = audio_exp.sort_values('test_acc', ascending=False)
columns = ['config', 'max_freq', 'test_acc', 'n_fft', 'keep_freq_point', 'hop_length','n_mels', 'resample_audio', 'extractor_type', 'nb_trainable_param_round', 'note']
exps = exps[columns]

grouped = exps.fillna(0).groupby(['config','max_freq','extractor_type', 'n_mels'], as_index=False).mean().sort_values('test_acc', ascending=False)
styler = color_by_multi_attribute(grouped, main_attribute='n_mels', attributes=['extractor_type', 'resample_audio', 'nb_trainable_param_round'], cmaps=['RdBu', 'YlOrRd'], format_dict=format_dict)

color_by = 'max_freq'
#color_by = 'resample_audio'
#color_by = 'n_fft'

display(Markdown(f"# Color coded by {color_by.capitalize()}"))
styler

In [ ]:
#exps.fillna(0).groupby(['n_fft', 'keep_freq_point', 'resample_audio']).mean().sort_values('test_acc', ascending=False)
#exps.fillna(0).groupby(['nb_trainable_param_round', 'resample_audio']).mean().sort_values('test_acc', ascending=False)
#exps.fillna(0).groupby(['config', 'resample_audio']).mean().sort_values('test_acc', ascending=False)
grouped = exps.fillna(0).groupby(['config','max_freq']).mean().sort_values('test_acc', ascending=False)

In [ ]:
24000 / (256 // 2) * 256

In [ ]:
grouped.style.format(format_dict)

In [ ]:
color_by = 'nb_trainable_param_round'
display(Markdown(f"# Color coded by {color_by.capitalize()}"))
color_row_by_attribute(grouped, color_by, cmap=plt.get_cmap('YlOrRd'), format_dict=format_dict)

In [ ]:
styler = color_row_by_attribute(grouped2, attribute='test_acc', cmap=plt.get_cmap('YlOrRd_r'), format_dict=format_dict)
styler = styler.background_gradient(subset=['nb_trainable_param_round'])
styler = styler.background_gradient(cmap='BrBG', subset=['max_freq'])

styler

In [ ]:
grouped2 = exps.fillna(0).groupby(['config','max_freq'], as_index=False).mean().sort_values('test_acc', ascending=False)
grouped2

In [ ]:
cmap = plt.get_cmap('YlOrRd')
cmap = None

exps = experiments[experiments['date'] >= '2020-08-17'].sort_values('date', ascending=False)
cols = run_info_columns + accuracy_columns + ['extractor_filters', 'extractor_projection_size', 'extractor_out_chan'] + ['n_fft', 'n_mels'] + ['random_seed', 'nb_trainable_param_round', 'note']

exps = exps.sort_values('test_acc', ascending=False)[cols]

color_by = 'extractor_projection_size'
display(Markdown(f"# Color coded by {color_by.capitalize()}"))
styler = color_row_by_attribute(exps, color_by, cmap=cmap, format_dict=format_dict)

exps.style.use(styler.export())
#styler.bar(subset=['train_acc'], color='#000000', axis=0, align='zero')

t = styler.export()




#color_row_by_attribute(latest, 'n_fft', {512:'red', 4096:'yellow'}).sort_values('test_acc', ascending=False).style.format(format_dict)

In [ ]:
color_row_by_attribute

In [ ]:
exps.style.background_gradient()

In [ ]:
len(latest['n_fft'].unique())

In [ ]:
y = plt.get_cmap('Blues')
y(200)[0]

In [ ]:
import seaborn as sns

def plot_colum_dist(df, colum, data_selectors, legend_title=None):
    legend_title='n_fft'

    plt.figure()
    for selector in data_selectors:    
        sns.distplot(df[selector['selector']][colum])
    plt.legend(labels=[d['title'] for d in data_selectors], title=legend_title)


data_selectors = [
    {'title': '512', 'selector': latest['n_fft'] == 512},
    {'title': '4096', 'selector': latest['n_fft'] == 4096}
]

plot_colum_dist(latest, 'test_acc', data_selectors, legend_title='n_fft')


In [ ]:
kind='hex'
kind='kde'
kind='scatter'

p = sns.jointplot(x='n_fft', y='test_acc', data=latest[latest['n_fft'] == 512], kind='kde', hue='n_fft', n_levels=5, hue_kws={"cmap": ["Blues", "Greens"]})

#p.ax_joint.scatter(x='n_fft', y='test_acc', data=latest, c='n_fft')

In [ ]:


latest

In [ ]:
pairplot_cols = ['test_acc', 'n_fft', 'nb_epoch_trained', 'nb_trainable_param_round']

sns.pairplot(latest[pairplot_cols], hue='n_fft', kind='scatter')
#sns.pairplot(latest)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
sns.violinplot(x='nb_trainable_param_round', y='test_acc', data=latest, hue='n_fft', split=True)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
sns.stripplot(x='n_fft', y='test_acc', data=latest)

In [ ]:
plt.figure()
sns.kdeplot(latest[['n_fft', 'test_acc']], shade=True, n_levels=5, cbar=True)
sns.scatterplot(x='n_fft', y='test_acc', data=latest, hue='n_fft')

In [ ]:
plt.figure()
sns.distplot()

In [ ]:
nfft_4096_exps = latest[latest['n_fft'] == 4096].sort_values('test_acc', ascending=False)
nfft_4096_exps.style.format(format_dict)

In [ ]:
t = experiments[audio_cols + ['date', 'train_time']].sort_values('test_acc', ascending=False)

#t = t[t['note'].isin(latest_experiment_notes)]

t.sort_values('date')

In [ ]:
experiments.sort_values('date', ascending=False)[audio_cols + ['train_time']]

In [ ]:
experiments[experiments['config'].str.contains("3_block_64_proj")].sort_values('test_acc', ascending=False)[exp_results_cols].style.format(format_dict)

In [ ]:
set(experiments['note'].values)

In [ ]:
#experiments[experiments['note'] == 'batching']['config'].values.tolist()
df_filter = (experiments['config'].isin(experiments[experiments['note'] == 'batching']['config'].values.tolist()))
df_filter &= (experiments['nb_scene'] == 50000) & (experiments['nb_q_per_scene'] == 4)
df_filter &= (experiments['note'] == 'batching') | (experiments['note'] == 'extractor')
df_filter |= ((experiments['config']=='reduction_original_rnn_4096') & (experiments['note'] == 'final_dropout'))
experiments[df_filter].drop_duplicates(subset=['config', 'nb_scene', 'nb_q_per_scene', 'note'], keep='first').sort_values('config')[exp_results_cols + ['folder']]
#experiments[experiments['note'] == 'batching'].sort_values('date', ascending=False)[exp_results_cols].style.format(format_dict)

In [ ]:
def get_delete_experiment_from_drive_script(df, dryrun=False):
    for f, d in df[['folder', 'date']].values:
        res_path = f"{f}/{d.strftime('%Y-%m-%d_%Hh%M')}"
        cmd = f"rclone delete Drive:result/training/{res_path} -P"
        if dryrun:
            cmd += " --dry-run"
        print(cmd)
        
get_delete_experiment_from_drive_script(experiments[experiments['note'] == 'resnet_noratio'], dryrun=False)

In [ ]:
experiments[((experiments['note'] == 'extractor') & (experiments['extractor_type'] == 'freq_time_interlaced'))].sort_values('test_acc', ascending=False)[exp_results_cols + ['extractor_type']].style.format(format_dict)

In [ ]:
def convert_cols_to_int2(df, columns):
    convert_dict = {col : int for col in columns}
    # For some reasons, this stopped working.. Can't convert float64 to Nullable Int... Now need to fill NaN value
    for col in columns:
        if df[col].isnull().any():
            df[col] = df[col].fillna(0)

    return df.astype(convert_dict)

df = groupby_mean(experiments[(experiments['note'] == 'extractor')], ['config', 'nb_sample'], ['best_val_acc', 'train_acc', 'test_acc', '0.6_at_epoch', '0.7_at_epoch', '0.8_at_epoch'], exp_results_cols, add_count_col=True)

df = convert_cols_to_int2(df, ['0.6_at_epoch', '0.7_at_epoch', '0.8_at_epoch'])

df.style.format(format_dict)

In [ ]:
print("Available configs :")
set(experiments['config'].values)

In [ ]:
len(experiments.index.values)

In [ ]:
exp_results[(exp_results['note'] == 'extractor')].sort_values(['nb_sample', 'test_acc']).style.format(format_dict)

In [ ]:
df_filter = (~exp_results['config'].str.contains('film_original')) & (~exp_results['config'].str.contains('freq_time')) & (~exp_results['config'].str.contains('extractor')) & (~exp_results['config'].str.contains('resblock'))
df_filter &= (exp_results['nb_sample'] == 200000) & (~exp_results['config'].str.contains('avg')) & (exp_results['stopped_early'] != 'stop_threshold')
experiments[exp_results_cols + ['rnn_state_size']][df_filter].groupby('rnn_state_size').mean().sort_values(['test_acc'])#.style.format(format_dict)
#exp_results[df_filter].sort_values(['test_acc']).style.format(format_dict)

In [ ]:
exp_results[(exp_results['stopped_early'] == 'stop_threshold') & (~exp_results['config'].str.contains('extractor'))]

In [ ]:
#exp_results[exp_results['nb_scene'] == 50000]
#exp_results[exp_results['nb_sample'] == 200000].sort_values(['nb_scene', 'nb_q_per_scene']).style.format(format_dict)
exp_results

In [ ]:
exp_results[(exp_results['nb_scene'] <= 10000) & (exp_results['config'].str.contains('reduction'))].sort_values('test_acc').style.format(format_dict)

In [ ]:
df_filter = ((experiments['config'].str.contains('reduction')) & (~experiments['config'].str.contains('stem')) & (~experiments['config'].str.contains('extractor')) & (~experiments['config'].str.contains('resblock')) & (~experiments['config'].str.contains('avg'))) | ((experiments['config'] == 'film_original') & (exp_results['nb_scene'] == 20000) & (exp_results['nb_q_per_scene'] == 20))
reduction_experiments = experiments[experiments['config'].str.contains('reduction')][exp_results_cols + timing_columns].sort_values('config')
reduction_experiments#[reduction_experiments['stopped_early'] == 'NO']['folder'].values

In [ ]:
exp_results[(exp_results['config'].str.contains('interlaced_smallest'))].sort_values('test_acc', ascending=True).style.format(format_dict)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

def plot_training_stats(dataframe, names=None, title=None, colormap=cm.viridis, ax=None):
    nb_rows = len(dataframe.index)

    colorlist = {key: colors.rgb2hex(colormap(i)) for key, i in
                 zip(dataframe.index, np.linspace(0, 0.9, nb_rows))}

    if title is None:
        title = "Training Stats"
    
    if ax is None:
        fig, axs = plt.subplots(2,1)
        
    fig.suptitle(title)
    axs[0].set_title("Accuracy")
    axs[1].set_title("Loss")
        
    for row in dataframe.itertuples():
        label = f"{row.config} - {row.nb_scene//1000}k_{row.nb_q_per_scene}_q"
        lines = axs[0].plot(row.all_train_acc, label=f"TRAIN - {label}")
        axs[0].plot(row.all_val_acc, color=lines[0].get_color(), linestyle="--")
        
        axs[1].plot(row.all_train_loss, color=lines[0].get_color())
        axs[1].plot(row.all_val_loss, label=f"VAL - {label}", color=lines[0].get_color(), linestyle="--")
        
    # Draw legend out of the plot
    for ax in axs:
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.5, box.height])

        ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

    fig.tight_layout()
    
    return fig, ax

    #for key, group in dataframe.groupby(group_key):
    #    group.plot.scatter(ax=ax, x=x_axis, y=y_axis, c=colorlist[key], label=key, title=title)
    
with_training_stats = experiments[exp_results_cols + all_training_stats]
fig, ax = plot_training_stats(with_training_stats[(with_training_stats['config'].str.contains('separated'))])

In [ ]:
plot_training_stats(with_training_stats[with_training_stats['nb_q_per_scene'] == 1])

In [ ]:
# Visualization of test_acc by (nb_sample, nb_scene)
exp_results[(~exp_results['config'].str.contains('film_original')) & (~exp_results['config'].str.contains('extractor'))].hist(column='train_acc', by=['nb_sample', 'nb_scene'])


In [ ]:
e = experiments.sort_values('date', ascending=False)[exp_results_cols]
e[e['note'] != 'dataset_size_last']

In [ ]:


grouped_scatter(experiments, 'config', 'nb_sample', 'test_acc')

#grouped_scatter(experiments[experiments['config'] == 'film_original'], 'nb_q_per_scene', 'nb_scene', 'test_acc')

test = experiments.copy()

test['perf_delta_orig_vs_small_rnn'] = test[test['config'] == 'film_original']['test_acc'] - test[test['config'] == 'film_original_small_rnn']['test_acc']

In [ ]:
test = sub_cols_with_cond_and_create_new_col(experiments.copy(), 'perf_delta_orig_vs_small_rnn', 'test_acc', test['config'] == 'film_original_small_rnn', test['config'] == 'film_original', test['config'] == 'film_original_small_rnn')

test.sort_values(['nb_sample', 'nb_scene'], ascending=False)[run_info_columns + ['test_acc', 'perf_delta_orig_vs_small_rnn']]

In [ ]:
latex_table = experiments[experiments['config'] == 'film_original'][['nb_sample', 'nb_scene', 'nb_q_per_scene', 'train_acc', 'best_val_acc', 'test_acc', '0.8_at_epoch']].sort_values(['nb_sample', 'nb_scene']).fillna("None").to_latex(formatters=latex_format_dict)
print(latex_table)

# Table Generation

In [ ]:
## Network reduction -- GRU units
df_filter = (((experiments['note'] == 'final_dropout') | (experiments['note'] == 'final_last')) & (experiments['config'].str.contains('reduction')) & (~experiments['config'].str.contains('proj')) & (~experiments['config'].str.contains('conv')) & (experiments['classifier_type'] == 'fcn'))
columns = ['nb_trainable_param', 'rnn_state_size', 'train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned', 'config', 'random_seed']#, 'train_time']
#df = groupby_mean(experiments[(experiments['note'] == 'extractor')], ['config', 'nb_sample'], ['best_val_acc', 'train_acc', 'test_acc', '0.6_at_epoch', '0.7_at_epoch', '0.8_at_epoch'], exp_results_cols, add_count_col=True)

#df = convert_cols_to_int2(df, ['0.6_at_epoch', '0.7_at_epoch', '0.8_at_epoch'])

reduction_experiments = experiments[df_filter][columns]

# TODO : Add standard deviation
# TODO : Seems like the dataframe is not sorted... Is it because it's a groupby ??
reduction_experiments = groupby_mean(reduction_experiments, ['config'], ['best_val_acc', 'train_acc', 'test_acc'], columns, add_count_col=False)
reduction_experiments.sort_values('nb_trainable_param', ascending=False)
#reduction_experiments = experiments[df_filter][columns].sort_values('nb_trainable_param', ascending=False)
#print(reduction_experiments.to_latex(index=False, formatters=latex_format_dict))
reduction_experiments.style.format(latex_format_dict)

In [ ]:
## Network reduction -- Classifier
df_filter =  (experiments['note'] == 'final') & (experiments['config'].str.contains('reduction'))
df_filter &= (~experiments['config'].str.contains('extractor')) & (~experiments['config'].str.contains('proj'))
df_filter &= (experiments['rnn_state_size'].isin([4096, 1024, 256]))

columns = ['nb_trainable_param', 'rnn_state_size', 'classifier_type', 'classifier_conv_out', 'classifier_projection_out', 'train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_trained']#, 'train_time']
# This will drop duplicates and keep the biggest test_acc of the duplicates
#reduction_experiments = experiments[df_filter].sort_values('test_acc', ascending=False).drop_duplicates(subset=['config', 'nb_scene', 'nb_q_per_scene'], keep='first')
# Sort back via trainable params
reduction_experiments = experiments[df_filter]

# Cleanup

reduction_experiments = reduction_experiments.sort_values('nb_trainable_param', ascending=False)[columns]
print(reduction_experiments.to_latex(index=False, formatters=latex_format_dict))
#reduction_experiments.style.format(latex_format_dict)

In [ ]:
## Network reduction -- Reduction Filters/Nb Resblocks
df_filter =  (experiments['note'] == 'final') & (experiments['config'].str.contains('reduction'))
df_filter &= (experiments['config'].str.contains('extractor')) & (~experiments['config'].str.contains('proj'))
df_filter &= (experiments['rnn_state_size'].isin([4096, 1024, 256]))

only_rnn_reduction_filter = (experiments['note'] == 'final') & (experiments['config'].isin(['reduction_original_rnn_1024_fcn_no_conv_hidden_256', "reduction_original_rnn_1024_fcn_conv_256_hidden_512"]))

df_filter |= only_rnn_reduction_filter

columns = ['config', 'nb_trainable_param', 'extractor_out_chan', 'stem_out_chan', 'nb_resblock', 'classifier_conv_out', 'classifier_projection_out', 'train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned']#, 'train_time']
# This will drop duplicates and keep the biggest test_acc of the duplicates
#reduction_experiments = experiments[df_filter].sort_values('test_acc', ascending=False).drop_duplicates(subset=['config', 'nb_scene', 'nb_q_per_scene'], keep='first')
# Sort back via trainable params
reduction_experiments = experiments[df_filter]

#reduction_experiments = reduction_experiments.sort_values(['rnn_state_size', 'extractor_out_chan', 'stem_out_chan', 'nb_resblock', 'classifier_conv_out', 'classifier_projection_out'], ascending=False)[columns]
#reduction_experiments = reduction_experiments.sort_values('nb_trainable_param', ascending=False)[columns]
reduction_experiments = reduction_experiments.sort_values('test_acc', ascending=False)[columns]
#print(reduction_experiments.to_latex(index=False, formatters=latex_format_dict))
reduction_experiments.style.format(latex_format_dict)

In [ ]:
## Feature Extractor -- Parallel Extractor
df_filter = (experiments['note'] == 'extractor') & (experiments['extractor_type'] == 'freq_time_separated')

columns = ['mean_epoch_time','stem_out_chan', 'config','nb_trainable_param', 'extractor_nb_block', 'extractor_filters', 'extractor_projection_size', 'extractor_out_chan', 'train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned']#, 'train_time']

reduction_experiments = experiments[df_filter]

# Average Results grouped over ['config', 'nb_scene', 'nb_q_per_scene']
reduction_experiments = groupby_mean(reduction_experiments, 
                                     groupby_columns=['config', 'nb_scene', 'nb_q_per_scene'],
                                     mean_columns=['train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned'],
                                    selected_columns=columns,
                                    add_count_col=False)

reduction_experiments = convert_cols_to_int(reduction_experiments, ['nb_epoch_runned'])

reduction_experiments = reduction_experiments.sort_values('mean_epoch_time', ascending=False)#[columns]
#print(reduction_experiments.to_latex(index=False, formatters=latex_format_dict))
reduction_experiments.style.format(latex_format_dict)

In [ ]:
## Feature Extractor -- Interlaced Extractor -- Time First
df_filter = (experiments['note'] == 'extractor') & (experiments['extractor_type'] == 'freq_time_interlaced')
df_filter &= (experiments['config'].str.contains('timefirst'))

columns = ['nb_trainable_param', 'extractor_nb_block', 'extractor_filters', 'extractor_projection_size', 'extractor_out_chan', 'train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned']#, 'train_time']

reduction_experiments = experiments[df_filter]

#reduction_experiments.loc[:, 'time_first'] = experiments['config'].str.contains('timefirst')

# Average Results grouped over ['config', 'nb_scene', 'nb_q_per_scene']
reduction_experiments = groupby_mean(reduction_experiments, 
                                     groupby_columns=['config', 'nb_scene', 'nb_q_per_scene'],
                                     mean_columns=['train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned'],
                                    selected_columns=columns,
                                    add_count_col=False)

reduction_experiments = convert_cols_to_int(reduction_experiments, ['nb_epoch_runned'])

reduction_experiments = reduction_experiments.sort_values('test_acc', ascending=False)#[columns]
#print(reduction_experiments.to_latex(index=False, formatters=latex_format_dict))
reduction_experiments.style.format(latex_format_dict)

In [ ]:
## Feature Extractor -- Interlaced Extractor -- Freq First
df_filter = (experiments['note'] == 'extractor') & (experiments['extractor_type'] == 'freq_time_interlaced')
df_filter &= (~experiments['config'].str.contains('timefirst'))

columns = ['nb_trainable_param', 'extractor_nb_block', 'extractor_filters', 'extractor_projection_size', 'extractor_out_chan', 'train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned']#, 'train_time']

reduction_experiments = experiments[df_filter]

#reduction_experiments.loc[:, 'time_first'] = experiments['config'].str.contains('timefirst')

# Average Results grouped over ['config', 'nb_scene', 'nb_q_per_scene']
reduction_experiments = groupby_mean(reduction_experiments, 
                                     groupby_columns=['config', 'nb_scene', 'nb_q_per_scene'],
                                     mean_columns=['train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned'],
                                    selected_columns=columns,
                                    add_count_col=False)

reduction_experiments = convert_cols_to_int(reduction_experiments, ['nb_epoch_runned'])

reduction_experiments = reduction_experiments.sort_values('nb_trainable_param', ascending=False)#[columns]
print(reduction_experiments.to_latex(index=False, formatters=latex_format_dict))
#reduction_experiments.style.format(latex_format_dict)

In [ ]:
## Feature Extractor -- Interlaced Extractor -- Both Freq First and Time First
df_filter = (experiments['note'] == 'extractor') & (experiments['extractor_type'] == 'freq_time_interlaced')
#df_filter &= (~experiments['config'].str.contains('timefirst'))

columns = ['nb_trainable_param', 'first', 'extractor_nb_block', 'extractor_filters', 'extractor_projection_size', 'extractor_out_chan', 'train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned']#, 'train_time']

reduction_experiments = experiments[df_filter]

reduction_experiments.loc[:, 'first'] = experiments['config'].apply(lambda c: 'Time' if 'timefirst' in c else 'Frequency')

# Average Results grouped over ['config', 'nb_scene', 'nb_q_per_scene']
reduction_experiments = groupby_mean(reduction_experiments, 
                                     groupby_columns=['config', 'nb_scene', 'nb_q_per_scene'],
                                     mean_columns=['train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned'],
                                    selected_columns=columns,
                                    add_count_col=False)

reduction_experiments = convert_cols_to_int(reduction_experiments, ['nb_epoch_runned'])

reduction_experiments = reduction_experiments.sort_values(['first', 'nb_trainable_param'], ascending=False)#[columns]
print(reduction_experiments.to_latex(index=False, formatters=latex_format_dict))
#reduction_experiments.style.format(latex_format_dict)

In [ ]:
## Dataset size comparison -- Mixed -- 100k, 200k, 400k samples
import matplotlib

fig_name = "dataset_size_all_samples.pdf"
df_filter = (experiments['note'] == 'dataset_size')
columns = ['nb_sample', 'nb_scene', 'nb_q_per_scene', 'test_acc']
exp = experiments[df_filter][columns]
exp = exp.sort_values('nb_q_per_scene')


fig, ax = plt.subplots()
lines = []

grouped_by_sample = exp[exp['nb_sample'] == 400000]

# Get colorlist
group_unique_keys = grouped_by_sample['nb_scene'].unique()
colorlist = {key: colors.rgb2hex(matplotlib.cm.gist_rainbow_r(i)) for key, i in
             zip(group_unique_keys, np.linspace(0, 0.9, len(group_unique_keys)))}

# Plot 400k lines & markers
lines += ax.plot(grouped_by_sample['nb_q_per_scene'], grouped_by_sample['test_acc'], linewidth=1, linestyle=':', zorder=1)
grouped_scatter(grouped_by_sample, 'nb_scene', 'nb_q_per_scene', 'test_acc', ax = ax, show_label=True, colorlist=colorlist, 
                label_modifier=lambda n: f"{int(n/1000)}k scenes  ", additional_params={"marker": ",", "zorder":2, "edgecolor":lines[0].get_markerfacecolor(), "linewidth":1})

# Plot 200k lines & markers
grouped_by_sample = exp[exp['nb_sample'] == 200000]
lines += ax.plot(grouped_by_sample['nb_q_per_scene'], grouped_by_sample['test_acc'], linewidth=1, linestyle=':', zorder=1)
grouped_scatter(grouped_by_sample, 'nb_scene', 'nb_q_per_scene', 'test_acc', ax = ax, show_label=False, colorlist=colorlist, additional_params={"marker": ",", "zorder":2, "edgecolor":lines[1].get_markerfacecolor(), "linewidth":1})

# Plot 100k lines & markers
grouped_by_sample = exp[exp['nb_sample'] == 100000]
lines += ax.plot(grouped_by_sample['nb_q_per_scene'], grouped_by_sample['test_acc'], linewidth=1, linestyle=':', zorder=1)
grouped_scatter(grouped_by_sample, 'nb_scene', 'nb_q_per_scene', 'test_acc', ax = ax, show_label=False, colorlist=colorlist, additional_params={"marker": ",", "zorder":2, "edgecolor":lines[2].get_markerfacecolor(), "linewidth":1})

# Remove marker border from legend
for legend_handle in ax.get_legend().legendHandles:
    legend_handle.set_linewidths(0)

# Add Second legend
ax.add_artist(matplotlib.legend.Legend(ax, lines, ['400k samples', '200k samples', '100k samples'], loc='center right'))

# Set axis infos
ax.set_xscale('log')
ax.set_xticks(exp['nb_q_per_scene'].unique())
ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
ax.set_xlim([0.9, 43])
ax.set_xlabel('Number of question per scene')
ax.set_ylabel('Accuracy')

fig.savefig(f"stats/{fig_name}", bbox_inches='tight')

In [ ]:
## Batching -- Pad per batch
df_filter = experiments['note'] == 'batching_real'

columns = ['config', 'nb_trainable_param', 'extractor_type', 'extractor_nb_block', 'extractor_filters', 'extractor_projection_size', 'extractor_out_chan', 'train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned']#, 'train_time']

reduction_experiments = experiments[df_filter]

reduction_experiments = reduction_experiments.sort_values('nb_trainable_param', ascending=False)#[columns]
#print(reduction_experiments.to_latex(index=False, formatters=latex_format_dict))
reduction_experiments[columns].style.format(latex_format_dict)

In [ ]:
config_used_in_batching_exp = experiments[experiments['note'] == 'batching_real']['config'].values.tolist()
df_filter = (experiments['config'].isin(config_used_in_batching_exp))
df_filter &= (experiments['nb_scene'] == 50000) & (experiments['nb_q_per_scene'] == 4)
#df_filter &= (~pd.isnull(experiments['note']))
df_filter &= (experiments['note'].isin(['batching_real', 'final_dropout', 'extractor']))

exp = experiments[df_filter]

exp = exp.sort_values('test_acc', ascending=False).drop_duplicates(['nb_scene', 'nb_q_per_scene', 'config', 'note'], keep='first')

#exp['padding'] = "Whole set" if exp['note'] == "batching_real" else "Per batch"
exp['batching'] = exp['note'].apply(lambda x: "Pad to set" if x == "batching_real" else "Pad to batch")

exp[exp_results_cols + ['batching']].sort_values('config')

#experiments[df_filter][exp_results_cols + ['random_seed']].sort_values('config', ascending=False)

In [ ]:
## More reduction with freq_time extractor
df_filter = (experiments['note'] == 'reduction_extractor')
df_filter |= (experiments['config'] == 'reduction_original_rnn_1024_fcn_conv_256_hidden_512_extractor_32_stem_32_resblock_3')


columns = ['config', 'nb_trainable_param', 'extractor_type', 'extractor_nb_block', 'extractor_filters', 'extractor_projection_size', 'extractor_out_chan', 'stem_out_chan', 'train_acc', 'best_val_acc', 'test_acc', 'nb_epoch_runned']#, 'train_time']

reduction_experiments = experiments[df_filter]

reduction_experiments = reduction_experiments.sort_values('nb_trainable_param', ascending=False)#[columns]
#print(reduction_experiments.to_latex(index=False, formatters=latex_format_dict))
reduction_experiments[columns].style.format(latex_format_dict)